In [8]:
## TODO 
# Set username & blog
base_url = 'http://www.nytimes.com/column/paul-krugman'
username = 'innocuousegg0'

## Absolute path of chromedriver
chromedriver = "/Users/elaine/Documents/twitbots/chromedriver"

In [2]:
import requests 
from bs4 import BeautifulSoup

In [3]:
import json

In [4]:
# For sites that block scrapers
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os

os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
urls = []

# ft alphaville
for i in range(1,250):
    url = base_url + str(i)
    r = requests.get(url)
    articles = BeautifulSoup(r.content).findAll('h3')
    for art in articles:
        try:
            href = art.find('a')['href']
            urls.append(href)
        except:
            continue


In [ ]:
blogs = {}
# require http referer for FT
s = requests.Session()
s.headers.update({'referer': 'http://www.google.com/'})

for url in urls:
    r = s.get(url)
    content = BeautifulSoup(r.content).find('div',{'class': 'entry-content'})



In [6]:
## Most wordpress blogs
##

urls = []
blogs = {}
s = requests.Session()

for i in range(1,91):
    url = base_url + str(i)
    req = requests.Request('GET', url, 
                         headers={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'})
    prepped = req.prepare()
    r = s.send(prepped)
    articles = BeautifulSoup(r.content).findAll('article', {'class': 'post'})
    for art in articles:
        ent = art.find('h1',{'class': 'entry-title'})
        href = ent.find('a')['href']
        urls.append(href)
        content = art.find('div',{'class': 'entry-content'})
        text = ''
        p = 0
        for t in content.findAll('p'):
            if t.get('style') and "padding-left: 30px" in t.get('style'):
                continue
            blogs[href+str(p)] = t.getText()
            p+=1

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [9]:
# Older Wordpress blogs

urls = []
blogs = {}

for i in range(1,38):
    url = base_url + str(i)
    r = requests.get(url)
    articles = BeautifulSoup(r.content).findAll('div',{'class': 'post'})
    for art in articles:
        try:
            ent = art.find('h2',{'class': 'posttitle'})
            href = ent.find('a')['href']
            urls.append(href)
            content = art.find('div',{'class': 'entry'})
            p = 0
            for t in content.findAll('p'):
                blogs[href+str(p)] = t.getText()
                p+=1
        except:
            print art
            continue

In [87]:
## NYtimes
##
browser = webdriver.Chrome(chromedriver)

url = base_url

browser.get(url)
time.sleep(1)

elem = browser.find_element_by_tag_name("body")
button = browser.find_element_by_class_name("load-more-button").click()

no_of_pagedowns = 0

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    no_of_pagedowns-=1

post_elems = browser.find_elements_by_class_name("story-link")

urls=[]
for post in post_elems:
    urlclean = post.get_attribute('href').split('?')
    urls.append(urlclean[0])
    
print urls[-1]

http://www.nytimes.com/2015/05/11/opinion/paul-krugman-wall-street-vampires.html


In [ ]:
## NYtimes continued
##

blogs = {}

for url in urls:    
    r = requests.get(url)
    content = BeautifulSoup(r.content).find('div',{'class': 'entry-content'})
    text=''
    try:
        for t in content.findAll('p',{'class': 'story-body-text'}):
             text += t.getText()
        blogs[url] = text
    except:
        print url
        continue

In [10]:
# Dump file

with open('data/%s-urls.json' % username, 'w') as f:
    json.dump(urls, f)
with open('data/%s-blog.json' % username, 'w') as f:
    json.dump(blogs, f)